In [1]:
# -*- coding:UTF-8 -*-
#process image
import keras, cv2,os
import numpy as np
from keras.utils import to_categorical
def read_image(imageName):
    img=cv2.imread(imageName,cv2.IMREAD_COLOR)
    #method1
    if img is None:
            return None;
    else:
        img = cv2.resize(img,dsize=(128,128),interpolation=cv2.INTER_LINEAR)
        img = img.astype("float32")
        img *= (1./255)
        b,g,r=cv2.split(img)
        img2=cv2.merge([r,g,b])
        return img2
        
def img_processing(path,x,y):
    directory =os.listdir(path)
    for textPath in directory:
        for fn in os.listdir(os.path.join(r"",path+ textPath)):
#             if fn.endswith('.png') or fn.endswith('.jpg'):
            fd = os.path.join(path, textPath, fn)
            img_arr=read_image(fd)
            if(img_arr is not None):
                x.append(img_arr)
                if(textPath=="ham"):
                    y.append(0)
                else:
                    y.append(1)
    return x,y  
x,y=[],[]
# get image email dataset path
path1="/home/aqts/yangHong/first-spam-experiment/data/image_email_dataset/"

x,y=img_processing(path1,x,y)
# VALIDATION_SPLIT=0.8


data = np.array(x)

labels = to_categorical(np.asarray(y))
print('Shape of data tensor:', data.shape)
print('Shape of label tensor:', labels.shape)

# split the data into a training set and a validation set
indices = np.arange(data.shape[0])
np.random.shuffle(indices)
data = data[indices]
labels = labels[indices]
# num_validation_samples = int( VALIDATION_SPLIT* data.shape[0])
# val_num=num_validation_samples+int(0.2* data.shape[0])
x_train = data[:]
y_train = labels[:]
# x_valid = data[num_validation_samples:val_num]
# y_valid = labels[num_validation_samples:val_num]
# x_test = data[num_validation_samples:]
# y_test = labels[num_validation_samples:]

print('x_train.shape',x_train.shape)
print('y_train.shape',y_train.shape)
# print('x_valid.shape',x_valid.shape)
# print('y_valid.shape',y_valid.shape)
# print('x_test.shape',x_test.shape)
# print('y_test.shape',y_test.shape)

Using TensorFlow backend.


Shape of data tensor: (2786, 128, 128, 3)
Shape of label tensor: (2786, 2)
x_train.shape (2786, 128, 128, 3)
y_train.shape (2786, 2)


In [2]:
# -*- coding:UTF-8 -*-
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers.convolutional import Convolution2D, MaxPooling2D, ZeroPadding2D,Conv2D,AveragePooling2D
from keras.layers import Activation, Dropout, Flatten, Dense,BatchNormalization,advanced_activations,initializers
from keras import backend as K
from keras.callbacks import EarlyStopping
from PIL import Image,ImageFile
from keras import regularizers
import matplotlib.pyplot as plt
from keras.optimizers import SGD, Adam
from sklearn.model_selection import KFold
import os
from sklearn.metrics import classification_report,confusion_matrix
from sklearn.metrics import f1_score,recall_score
from sklearn.metrics import roc_curve,auc
ImageFile.LOAD_TRUNCATED_IMAGES = True
# use GPU block
# 随机数设置，便于产生相同的随机数
seed = 42
np.random.seed(seed)

# 定义 K-fold 交叉验证 参数
kfold= KFold(n_splits=5, shuffle=True, random_state=seed)

os.environ["CUDA_VISIBLE_DEVICES"] = "0"
# dimensions of our images.
img_width, img_height = 128,128
if K.image_data_format() == 'channels_first':
    input_shape = (3, img_width, img_height)
else:
    input_shape = (img_width, img_height, 3)
flag=1
n_classes=2
flag=1
f1_scores=[]
recall_scores=[]
cvscores=[]
for train, test in kfold.split(x_train, y_train):
    # Cross validation 交叉验证结果
    model = Sequential()
    model.add(Conv2D(filters=32, kernel_size=(5,5), padding='same', input_shape=input_shape))
    model.add(BatchNormalization(axis=-1, momentum=0.9, epsilon=1e-6))
    model.add(Activation("relu"))
    model.add(MaxPooling2D(pool_size=(2,2)))

    model.add(Conv2D(filters=64, kernel_size=(5,5), padding='same'))
    model.add(BatchNormalization(axis=-1, momentum=0.9, epsilon=1e-6))
    model.add(Activation("relu"))
    model.add(MaxPooling2D(pool_size=(2,2)))

    model.add(Conv2D(filters=64, kernel_size=(5,5), padding='same'))
    model.add(BatchNormalization(axis=-1, momentum=0.9, epsilon=1e-6))
    model.add(Activation("relu"))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Flatten())
    model.add(Dense(64))
    model.add(BatchNormalization(epsilon=1e-6))
    model.add(Activation("relu"))

    model.add(Dense(32, activation="relu"))
    model.add(Dense(2, activation='softmax'))
    # adam = Adam(lr=0.001, beta_1=0.79, beta_2=0.999, epsilon=1e-8) 
    # sgd =SGD(lr=0.01, decay=1e-6, momentum=0.89, nesterov=True)
    model.compile(optimizer="sgd", loss='binary_crossentropy', metrics=["accuracy"])
    #model.summary()
    model.fit(x_train[train], y_train[train], epochs=32, batch_size=20, verbose=1)
    model.save("/home/aqts/yangHong/first-spam-experiment/h5_model/image_cnn_model_k_flod"+str(flag)+".h5")
    # 评估模型
    # accuracy
    scores = model.evaluate(x_train[test], y_train[test], verbose=0)
#     print("%s: %s %.2f%%" % (model.metrics_names[1], ':', scores[1]*100))
    cvscores.append(scores[1]*100)
    y_pred_score=y_pred=model.predict(x_train[test], 32)
    #get precision
    for i in range(len(y_pred)):
        if(y_pred[i][0]>0.5):
            y_pred[i][0]=1
        else:
            y_pred[i][0]=0
        if(y_pred[i][1]>0.5):
            y_pred[i][1]=1
        else:
            y_pred[i][1]=0
    print("---------------------------Precision-----------------------------")
    print(classification_report(y_train[test],y_pred))
    #f1-score
    f1_scores.append(f1_score(y_train[test],y_pred,average = 'macro'))
    #recall-score
    recall_scores.append(recall_score(y_train[test],y_pred,average = 'macro'))
    #get auc
    fpr = dict()
    tpr = dict()
    roc_auc = dict()
    for i in range(n_classes):
        fpr[i], tpr[i], _ = roc_curve(y_train[test][:, i], y_pred_score[:, i])    
        roc_auc[i] = auc(fpr[i], tpr[i])   

    # #Plot of a ROC curve for a specific class
    plt.rcParams['figure.figsize']=(8,5)
    plt.figure()
    plt.plot(fpr[1], tpr[1], color='darkorange', label='ROC curve spam(auc = %0.4f)' % roc_auc[1])
    plt.plot(fpr[0], tpr[0], color='green', label='ROC curve ham (auc = %0.4f)' % roc_auc[0])
    plt.plot([0, 1], [0, 1], color='navy', linestyle='--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('ROC graph for predicting image dataset using the MMA-MF model')
    plt.legend(loc="lower right")
    plt.savefig("/home/aqts/yangHong/first-spam-experiment/experiment_result_roc/ROC_graph_for_Image"+str(flag)+".png")
    plt.show()
    flag=flag+1

print("***************************Accuracy****************************")
print("The all of acc score: " ,(cvscores))
print("The average score: %.2f%% (+/- %.2f%%)" % (np.mean(cvscores), np.std(cvscores)))
print("---------------------------F1-Score-----------------------------")
print("The all of f1_scores score: " ,(f1_scores))
print("The average score: %.2f%% (+/- %.2f%%)" % (np.mean(f1_scores), np.std(f1_scores)))
print("+++++++++++++++++++++++++++Recall-Score+++++++++++++++++++++++++")
print("The all of f1_scores score: " ,(recall_scores))
print("The average score: %.2f%% (+/- %.2f%%)" % (np.mean(recall_scores), np.std(recall_scores)))


Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Epoch 1/32
2228/2228 [==============================] - 93s 42ms/step - loss: 0.4263 - acc: 0.8012
Epoch 2/32
2228/2228 [==============================] - 92s 41ms/step - loss: 0.2611 - acc: 0.8986
Epoch 3/32
2228/2228 [==============================] - 92s 41ms/step - loss: 0.1943 - acc: 0.9237
Epoch 4/32
2228/2228 [==============================] - 92s 41ms/step - loss: 0.1326 - acc: 0.9641
Epoch 5/32
2228/2228 [==============================] - 91s 41ms/step - loss: 0.0975 - acc: 0.9744
Epoch 6/32
2228/2228 [==============================] - 92s 41ms/step - loss: 0.0620 - acc: 0.9852
Epoch 7/32
2228/2228 [==============================] - 91s 41ms/step - loss: 0.0387 - acc: 0.9933
Epoch 8/32
2228/2228 [==============================] - 91s 41ms/step - loss: 0.0504 - acc: 0.9852
Epoch 9/32
2228/2228 [==============================] - 91s 41ms/step - loss: 0.0678 - a

<Figure size 800x500 with 1 Axes>

Epoch 1/32
2229/2229 [==============================] - 154s 69ms/step - loss: 0.3915 - acc: 0.8277
Epoch 2/32
2229/2229 [==============================] - 152s 68ms/step - loss: 0.2623 - acc: 0.8919
Epoch 3/32
2229/2229 [==============================] - 152s 68ms/step - loss: 0.1746 - acc: 0.9390
Epoch 4/32
2229/2229 [==============================] - 153s 69ms/step - loss: 0.1326 - acc: 0.9592
Epoch 5/32
2229/2229 [==============================] - 151s 68ms/step - loss: 0.0909 - acc: 0.9753
Epoch 6/32
2229/2229 [==============================] - 132s 59ms/step - loss: 0.0624 - acc: 0.9856
Epoch 7/32
2229/2229 [==============================] - 139s 62ms/step - loss: 0.0583 - acc: 0.9838
Epoch 8/32
2229/2229 [==============================] - 151s 68ms/step - loss: 0.0353 - acc: 0.9919
Epoch 9/32
2229/2229 [==============================] - 152s 68ms/step - loss: 0.0323 - acc: 0.9937
Epoch 10/32
2229/2229 [==============================] - 153s 69ms/step - loss: 0.0276 - acc: 0.9955

<Figure size 800x500 with 1 Axes>

Epoch 1/32
2229/2229 [==============================] - 222s 99ms/step - loss: 0.4140 - acc: 0.8080
Epoch 2/32
2229/2229 [==============================] - 191s 86ms/step - loss: 0.2669 - acc: 0.8914
Epoch 3/32
2229/2229 [==============================] - 111s 50ms/step - loss: 0.1837 - acc: 0.9358
Epoch 4/32
2229/2229 [==============================] - 110s 49ms/step - loss: 0.1199 - acc: 0.9668
Epoch 5/32
2229/2229 [==============================] - 116s 52ms/step - loss: 0.0884 - acc: 0.9771
Epoch 6/32
2229/2229 [==============================] - 102s 46ms/step - loss: 0.0761 - acc: 0.9771
Epoch 7/32
2229/2229 [==============================] - 91s 41ms/step - loss: 0.0545 - acc: 0.9838
Epoch 8/32
2229/2229 [==============================] - 89s 40ms/step - loss: 0.0410 - acc: 0.9924
Epoch 9/32
2229/2229 [==============================] - 104s 47ms/step - loss: 0.0411 - acc: 0.9870
Epoch 10/32
2229/2229 [==============================] - 96s 43ms/step - loss: 0.0230 - acc: 0.9960
Ep

...

Epoch 1/32
2229/2229 [==============================] - 92s 41ms/step - loss: 0.4164 - acc: 0.8017
Epoch 2/32
2229/2229 [==============================] - 90s 41ms/step - loss: 0.2700 - acc: 0.8901
Epoch 3/32
2229/2229 [==============================] - 92s 41ms/step - loss: 0.2003 - acc: 0.9233
Epoch 4/32
2229/2229 [==============================] - 90s 41ms/step - loss: 0.1456 - acc: 0.9515
Epoch 5/32
2229/2229 [==============================] - 91s 41ms/step - loss: 0.1126 - acc: 0.9619
Epoch 6/32
2229/2229 [==============================] - 90s 41ms/step - loss: 0.0769 - acc: 0.9803
Epoch 7/32
2229/2229 [==============================] - 90s 40ms/step - loss: 0.0497 - acc: 0.9888
Epoch 8/32
2229/2229 [==============================] - 91s 41ms/step - loss: 0.0465 - acc: 0.9883
Epoch 9/32
2229/2229 [==============================] - 91s 41ms/step - loss: 0.0265 - acc: 0.9933
Epoch 10/32
2229/2229 [==============================] - 91s 41ms/step - loss: 0.0339 - acc: 0.9910
Epoch 11/

<Figure size 800x500 with 1 Axes>

Epoch 1/32
2229/2229 [==============================] - 167s 75ms/step - loss: 0.4148 - acc: 0.8035
Epoch 2/32
2229/2229 [==============================] - 161s 72ms/step - loss: 0.2713 - acc: 0.8905
Epoch 3/32
2229/2229 [==============================] - 161s 72ms/step - loss: 0.1887 - acc: 0.9323
Epoch 4/32
2229/2229 [==============================] - 156s 70ms/step - loss: 0.1357 - acc: 0.9592
Epoch 5/32
2229/2229 [==============================] - 160s 72ms/step - loss: 0.0901 - acc: 0.9762
Epoch 6/32
2229/2229 [==============================] - 158s 71ms/step - loss: 0.0719 - acc: 0.9794
Epoch 7/32
2229/2229 [==============================] - 160s 72ms/step - loss: 0.0487 - acc: 0.9874
Epoch 8/32
2229/2229 [==============================] - 182s 82ms/step - loss: 0.0445 - acc: 0.9856
Epoch 9/32
2229/2229 [==============================] - 228s 102ms/step - loss: 0.0411 - acc: 0.9861
Epoch 10/32
2229/2229 [==============================] - 181s 81ms/step - loss: 0.0365 - acc: 0.992

<Figure size 800x500 with 1 Axes>

***************************Accuracy****************************
The all of acc score:  [92.63913825127553]
The average score: 92.64% (+/- 0.00%)
---------------------------F1-Score-----------------------------
The all of f1_scores score:  [0.9262544684231431]
The average score: 0.93% (+/- 0.00%)
+++++++++++++++++++++++++++Recall-Score+++++++++++++++++++++++++
The all of f1_scores score:  [0.9263235408107899]
The average score: 0.93% (+/- 0.00%)


In [5]:
print("***************************Accuracy****************************")
print("The all of acc score: " ,(cvscores))
print("The average score: %.4f%% (+/- %.4f%%)" % (np.mean(cvscores), np.std(cvscores)))
print("---------------------------F1-Score-----------------------------")
print("The all of f1_scores score: " ,(f1_scores))
print("The average score: %.4f%% (+/- %.4f%%)" % (np.mean(f1_scores), np.std(f1_scores)))
print("+++++++++++++++++++++++++++Recall-Score+++++++++++++++++++++++++")
print("The all of recall_scores score: " ,(recall_scores))
print("The average score: %.4f%% (+/- %.4f%%)" % (np.mean(recall_scores), np.std(recall_scores)))

***************************Accuracy****************************
The all of acc score:  [92.63913825127553]
The average score: 92.6391% (+/- 0.0000%)
---------------------------F1-Score-----------------------------
The all of f1_scores score:  [0.9262544684231431]
The average score: 0.9263% (+/- 0.0000%)
+++++++++++++++++++++++++++Recall-Score+++++++++++++++++++++++++
The all of recall_scores score:  [0.9263235408107899]
The average score: 0.9263% (+/- 0.0000%)
